# Load data 

首先请你学习plot_waterfall_with_subplot函数，熟悉瀑布图绘制的基本逻辑

其次请你学习analyze_policy_paths_by_pixel_with_ci函数，熟悉各个价格的基本逻辑

现在我需要按照不同政策类别绘制不同的瀑布图（不要P4、一共9个子图，每个图30mm的width，按照3×3排列）+Overall（含有P4）的图

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib

# Load data
df_economic = pd.read_csv('data/US_data/df_economic.csv')
df_pv_npv = pd.read_csv(r'data\US_data\df_pv_npv.csv')
df_agricultural_npv=pd.read_csv(r'data\US_data\df_agricultural_npv.csv')
df_afforestation_npv=pd.read_csv(r'data\US_data\df_afforestation_npv.csv')
df_natural_npv=pd.read_csv(r'data\US_data\df_natural_npv.csv')
# df_climate_zone=pd.read_csv(r'data\US_data\df_zones.csv')
df_weight=pd.read_csv(r'data\US_data\df_weight.csv')

# 定义政策类别（不包括P4）
policy_categories = ['P1a', 'P1b', 'P1c', 'P1d', 'P2a', 'P2c', 'P2', 'P3a', 'P3b', 'P3c']

print(f"Policy categories: {policy_categories}")
print(f"Total categories: {len(policy_categories)}")


Policy categories: ['P1a', 'P1b', 'P1c', 'P1d', 'P2a', 'P2c', 'P2', 'P3a', 'P3b', 'P3c']
Total categories: 10


In [16]:
def calculate_waterfall_data_by_policy_categories(
    df_pv_npv, df_agricultural_npv, df_afforestation_npv, 
    df_natural_npv, df_economic, df_weight, policy_categories, target_year=2050):
    """
    计算指定政策类别的瀑布图数据
    
    Parameters
    ----------
    df_pv_npv : DataFrame
        光伏NPV数据
    df_agricultural_npv : DataFrame
        农业NPV数据
    df_afforestation_npv : DataFrame
        造林NPV数据
    df_natural_npv : DataFrame
        自然恢复NPV数据
    df_economic : DataFrame
        经济可行性数据
    df_weight : DataFrame
        权重数据
    policy_categories : list
        政策类别列表，如 ['P1a', 'P1b', 'P1c', 'P1d', 'P2a', 'P2c', 'P2', 'P3a', 'P3b', 'P3c']
    target_year : int
        目标年份，默认2050
        
    Returns
    -------
    df_benefit : DataFrame
        包含所有政策类别的瀑布图数据
    df_opportunity : DataFrame
        各RCP类别下的机会成本期望值
    """
    import pandas as pd
    import numpy as np
    
    # 列名小写
    for df in [df_pv_npv, df_agricultural_npv, df_afforestation_npv, df_natural_npv, df_economic, df_weight]:
        df.columns = df.columns.str.lower()
    
    # 筛选目标年份数据
    pv_data = df_pv_npv[df_pv_npv['analysis_year'] == target_year].copy()
    agri_data = df_agricultural_npv[df_agricultural_npv['analysis_year'] == target_year].copy()
    aff_data = df_afforestation_npv[df_afforestation_npv['analysis_year'] == target_year].copy()
    natural_data = df_natural_npv[df_natural_npv['analysis_year'] == target_year].copy()
    economic_data = df_economic[df_economic['analysis_year'] == target_year].copy()
    
    # 合并权重数据
    def merge_with_weight(df, weight_cols):
        return df.merge(df_weight[['lat', 'lon'] + weight_cols], on=['lat', 'lon'], how='left')
    
    agri_weighted = merge_with_weight(agri_data, ['weighted_density_agricultural'])
    aff_weighted = merge_with_weight(aff_data, ['weighted_density_forest'])
    natural_weighted = merge_with_weight(natural_data, ['weighted_density_vegetation'])
    
    # ========== 新增：计算 df_opportunity ==========
    # 识别 RCP 列名（可能在 rcp_category, RCP_Category 等）
    def find_rcp_column(df):
        for col in df.columns:
            if 'rcp' in col.lower() and 'category' in col.lower():
                return col
        return None
    
    rcp_col_agri = find_rcp_column(agri_data)
    rcp_col_aff = find_rcp_column(aff_data)
    rcp_col_natural = find_rcp_column(natural_data)
    
    # 准备机会成本数据
    opportunity_results = []
    
    if rcp_col_agri and rcp_col_aff and rcp_col_natural:
        # 获取所有唯一的 RCP 类别
        all_rcps = sorted(set(agri_weighted[rcp_col_agri].dropna().unique()) | 
                          set(aff_weighted[rcp_col_aff].dropna().unique()) | 
                          set(natural_weighted[rcp_col_natural].dropna().unique()))
        
        print(f"\n发现 RCP 类别: {all_rcps}")
        
        # 为每个 RCP 类别计算期望值
        for rcp in all_rcps:
            # 筛选该 RCP 的数据
            agri_rcp = agri_weighted[agri_weighted[rcp_col_agri] == rcp].copy()
            aff_rcp = aff_weighted[aff_weighted[rcp_col_aff] == rcp].copy()
            natural_rcp = natural_weighted[natural_weighted[rcp_col_natural] == rcp].copy()
            
            if not agri_rcp.empty and not aff_rcp.empty and not natural_rcp.empty:
                # 像素级别计算
                agri_pixel_rcp = agri_rcp.groupby(['lat', 'lon'], as_index=False)[['cumulative_npv_usd', 'weighted_density_agricultural']].mean()
                aff_pixel_rcp = aff_rcp.groupby(['lat', 'lon'], as_index=False)[['cumulative_npv_usd', 'weighted_density_forest']].mean()
                natural_pixel_rcp = natural_rcp.groupby(['lat', 'lon'], as_index=False)[['cumulative_npv_usd', 'weighted_density_vegetation']].mean()
                
                # 合并数据
                merged_rcp = agri_pixel_rcp.merge(aff_pixel_rcp, on=['lat', 'lon'], how='inner', suffixes=('', '_aff'))
                merged_rcp = merged_rcp.merge(natural_pixel_rcp, on=['lat', 'lon'], how='inner', suffixes=('', '_natural'))
                
                # 计算各组成部分的期望值
                agricultural_expected = (merged_rcp['weighted_density_agricultural'] * merged_rcp['cumulative_npv_usd']).mean()
                forest_expected = (merged_rcp['weighted_density_forest'] * merged_rcp['cumulative_npv_usd_aff']).mean()
                vegetation_expected = (merged_rcp['weighted_density_vegetation'] * merged_rcp['cumulative_npv_usd_natural']).mean()
                
                opportunity_results.append({
                    'RCP_Category': rcp,
                    'Agricultural_Expected': agricultural_expected,
                    'Forest_Expected': forest_expected,
                    'Vegetation_Expected': vegetation_expected,
                    'Total_Expected': agricultural_expected + forest_expected + vegetation_expected
                })
    
    df_opportunity = pd.DataFrame(opportunity_results)
    
    # 存储所有结果
    results = []
    
    # 为每个政策类别计算
    for policy in policy_categories:
        # 筛选该政策的数据
        pv_subset = pv_data[pv_data['policy_category'] == policy].copy()
        economic_subset = economic_data[economic_data['policy_category'] == policy].copy()
        
        # 1. Revenue (收入) - cumulative_npv_usd + cumulative_cost_usd
        if not pv_subset.empty:
            pv_grouped = pv_subset.groupby(['lat', 'lon'], as_index=False)[['cumulative_npv_usd', 'cumulative_cost_usd']].mean()
            revenue_mean = (pv_grouped['cumulative_npv_usd'] + pv_grouped['cumulative_cost_usd']).mean()
        else:
            revenue_mean = 0
        
        # 2. Expenditure (支出) - cumulative_cost_usd
        if not pv_subset.empty:
            pv_grouped = pv_subset.groupby(['lat', 'lon'], as_index=False)['cumulative_cost_usd'].mean()
            expenditure_mean = pv_grouped['cumulative_cost_usd'].mean()
        else:
            expenditure_mean = 0
        
        # 3. Opportunity (机会成本) - 使用所有像素的平均值
        agri_pixel = agri_weighted.groupby(['lat', 'lon'], as_index=False)[['cumulative_npv_usd', 'weighted_density_agricultural']].mean()
        aff_pixel = aff_weighted.groupby(['lat', 'lon'], as_index=False)[['cumulative_npv_usd', 'weighted_density_forest']].mean()
        natural_pixel = natural_weighted.groupby(['lat', 'lon'], as_index=False)[['cumulative_npv_usd', 'weighted_density_vegetation']].mean()
        
        # 合并数据
        merged = agri_pixel.merge(aff_pixel, on=['lat', 'lon'], how='inner', suffixes=('', '_aff'))
        merged = merged.merge(natural_pixel, on=['lat', 'lon'], how='inner', suffixes=('', '_natural'))
        
        # 计算像素级别的机会成本
        merged['pixel_opportunity_cost'] = (
            merged['weighted_density_forest'] * merged['cumulative_npv_usd_aff'] +
            merged['weighted_density_agricultural'] * merged['cumulative_npv_usd'] +
            merged['weighted_density_vegetation'] * merged['cumulative_npv_usd_natural']
        )
        
        opportunity_mean = merged['pixel_opportunity_cost'].mean()
        
        # 4. Net (净收益) - net_npv_usd
        if not economic_subset.empty:
            economic_grouped = economic_subset.groupby(['lat', 'lon'], as_index=False)['net_npv_usd'].mean()
            net_mean = economic_grouped['net_npv_usd'].mean()
        else:
            net_mean = 0
        
        # 记录结果
        results.append({
            'policy_category': policy,
            'Revenue_Mean': revenue_mean,
            'Expenditure_Mean': expenditure_mean,
            'Opportunity_Mean': opportunity_mean,
            'Net_Mean': net_mean
        })
    
    # 计算 Overall (包含所有P1, P2, P3政策，P4也包含)
    pv_overall = pv_data.copy()
    economic_overall = economic_data.copy()
    
    # Revenue Overall
    if not pv_overall.empty:
        pv_overall_grouped = pv_overall.groupby(['lat', 'lon'], as_index=False)[['cumulative_npv_usd', 'cumulative_cost_usd']].mean()
        revenue_overall = (pv_overall_grouped['cumulative_npv_usd'] + pv_overall_grouped['cumulative_cost_usd']).mean()
    else:
        revenue_overall = 0
    
    # Expenditure Overall
    if not pv_overall.empty:
        pv_overall_grouped = pv_overall.groupby(['lat', 'lon'], as_index=False)['cumulative_cost_usd'].mean()
        expenditure_overall = pv_overall_grouped['cumulative_cost_usd'].mean()
    else:
        expenditure_overall = 0
    
    # Opportunity Overall (使用所有像素)
    opportunity_overall = merged['pixel_opportunity_cost'].mean()
    
    # Net Overall
    if not economic_overall.empty:
        economic_overall_grouped = economic_overall.groupby(['lat', 'lon'], as_index=False)['net_npv_usd'].mean()
        net_overall = economic_overall_grouped['net_npv_usd'].mean()
    else:
        net_overall = 0
    
    # 添加 Overall
    results.append({
        'policy_category': 'Overall',
        'Revenue_Mean': revenue_overall,
        'Expenditure_Mean': expenditure_overall,
        'Opportunity_Mean': opportunity_overall,
        'Net_Mean': net_overall
    })
    
    # 转换为 DataFrame
    df_benefit = pd.DataFrame(results)
    
    return df_benefit, df_opportunity


# 使用示例
policy_categories = ['P1a', 'P1b', 'P1c', 'P1d', 'P2a', 'P2c', 'P2', 'P3a', 'P3b', 'P3c']

df_benefit_for_waterfall, df_opportunity = calculate_waterfall_data_by_policy_categories(
    df_pv_npv, df_agricultural_npv, df_afforestation_npv, 
    df_natural_npv, df_economic, df_weight, 
    policy_categories, target_year=2050
)

print("\n=== Waterfall Data by Policy Categories ===")
print(df_benefit_for_waterfall)

print("\n=== Opportunity Cost by RCP Category ===")
print(df_opportunity)


发现 RCP 类别: ['RCP2.6', 'RCP4.5', 'RCP6.0', 'RCP8.5']

=== Waterfall Data by Policy Categories ===
   policy_category  Revenue_Mean  Expenditure_Mean  Opportunity_Mean  \
0              P1a  2.754030e+06      5.444369e+06       -799.336632   
1              P1b  3.291713e+06      5.497964e+06       -799.336632   
2              P1c  4.989486e+06      5.243445e+06       -799.336632   
3              P1d  2.637316e+06      4.171956e+06       -799.336632   
4              P2a  5.343923e+06      5.543635e+06       -799.336632   
5              P2c  3.732909e+06      5.866036e+06       -799.336632   
6               P2  3.321407e+06      4.225846e+06       -799.336632   
7              P3a  3.611629e+06      5.381052e+06       -799.336632   
8              P3b  7.229343e+06      5.188424e+06       -799.336632   
9              P3c  3.137213e+06      2.959927e+06       -799.336632   
10         Overall  5.161458e+06      5.310492e+06       -799.336632   

        Net_Mean  
0  -2.689539e+06  

In [6]:
df_benefit_for_waterfall

,policy_category,Revenue_Mean,Expenditure_Mean,Opportunity_Mean,Net_Mean
0,P1a,2.754030e+06,5.444369e+06,-799.336632,-2.689539e+06
1,P1b,3.291713e+06,5.497964e+06,-799.336632,-2.205451e+06
2,P1c,4.989486e+06,5.243445e+06,-799.336632,-2.531592e+05
3,P1d,2.637316e+06,4.171956e+06,-799.336632,-1.533840e+06
4,P2a,5.343923e+06,5.543635e+06,-799.336632,-1.989135e+05
5,P2c,3.732909e+06,5.866036e+06,-799.336632,-2.132328e+06
6,P2,3.321407e+06,4.225846e+06,-799.336632,-9.036400e+05
7,P3a,3.611629e+06,5.381052e+06,-799.336632,-1.768623e+06
8,P3b,7.229343e+06,5.188424e+06,-799.336632,2.041719e+06
9,P3c,3.137213e+06,2.959927e+06,-799.336632,1.780860e+05
